In [1]:
import sys
sys.path.append('/Users/tunadorable/local-repos/ng-video-lecture/venv/lib/python3.11/site-packages')

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import time

In [3]:
device = 'mps' if torch.backends.mps.is_available() else 'cpu'

In [18]:
# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 64 # what is the maximum context length for predictions?
max_iters = 10000
eval_interval = 500
learning_rate = 3e-4
eval_iters = 20
n_embd = 128
n_head = 8
n_layer = 8
dropout = 0.2

In [5]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [6]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(chars, vocab_size)

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'] 65


In [7]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [8]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [9]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [10]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval() # sets model to eval mode
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train() # just resets to training mode
    return out

In [11]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [12]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        super().__init__()
        self.sa = nn.MultiheadAttention(embed_dim=n_embd, num_heads=n_head)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self._sa_forward(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

    def _sa_forward(self, x):
        # MultiheadAttention expects inputs in (t, b, d) format.
        x = x.transpose(0, 1)  # Convert (b, t, d) to (t, b, d)
        attn_output, _ = self.sa(x, x, x)
        return attn_output.transpose(0, 1)  # Convert back to (b, t, d)

In [13]:
class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd) # learnable positional encoding
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        b, t = idx.shape

        # idx and targets are both (b,t) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (b,t,d)
        pos_emb = self.position_embedding_table(torch.arange(t, device=device)) # (t,d)
        x = tok_emb + pos_emb # (b,t,d) + (t,d) = (b,t,d)
        x = self.blocks(x) # (b,t,d) -> (b,t,d)
        x = self.ln_f(x) # (b,t,d) -> (b,t,d)
        
        # Use the transposed embedding matrix for the output layer
        logits = x @ self.token_embedding_table.weight.t() # (b,t,d)@(d,v)=(b,t,v)
        
        if targets is None:
            loss = None
        else:
            b, t, v = logits.shape
            logits = logits.view(b*t, v)
            targets = targets.view(b*t)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (b, t) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (b, d)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (b, d)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (b, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (b, t+1)
        return idx

In [20]:
model = GPTLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e3, 'K parameters')

1602.944 K parameters


In [21]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [22]:
start_time = time.time()
for iter in range(max_iters):

    # sample a batch of data
    xb, yb = get_batch('train')
    
    # train
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        current_time = time.time()
        elapsed_time = current_time - start_time
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}, time elapsed: {elapsed_time:.2f} seconds")

step 0: train loss 3.9566, val loss 3.9692, time elapsed: 0.77 seconds
step 500: train loss 0.1064, val loss 0.1088, time elapsed: 54.96 seconds
step 1000: train loss 0.0537, val loss 0.0506, time elapsed: 108.17 seconds
step 1500: train loss 0.0460, val loss 0.0445, time elapsed: 160.74 seconds
step 2000: train loss 0.0415, val loss 0.0445, time elapsed: 213.08 seconds
step 2500: train loss 0.0426, val loss 0.0402, time elapsed: 265.17 seconds
step 3000: train loss 0.0404, val loss 0.0387, time elapsed: 317.55 seconds
step 3500: train loss 0.0396, val loss 0.0383, time elapsed: 370.78 seconds
step 4000: train loss 0.0377, val loss 0.0374, time elapsed: 422.79 seconds
step 4500: train loss 0.0359, val loss 0.0371, time elapsed: 475.04 seconds
step 5000: train loss 0.0370, val loss 0.0388, time elapsed: 528.07 seconds
step 5500: train loss 0.0346, val loss 0.0349, time elapsed: 580.21 seconds
step 6000: train loss 0.0336, val loss 0.0350, time elapsed: 631.84 seconds
step 6500: train lo

In [ ]:
# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 64 # what is the maximum context length for predictions?
max_iters = 10000
eval_interval = 500
learning_rate = 3e-4
eval_iters = 20
n_embd = 128
n_head = 8
n_layer = 8
dropout = 0.2

In [23]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=64)[0].tolist()))


P

PPE

jjeEPA
A
P
E
jPPjjjj
d
POee
P
Edzejjj
EDPejejjeyjeey
jet


In [22]:
print("logits: ", logits.shape, logits)

logits:  torch.Size([1024, 65]) tensor([[-0.6153,  0.7349,  1.0283,  ..., -0.4749, -0.9604,  0.3338],
        [ 0.7572, 10.3071, -1.1817,  ..., -1.7752, -2.9931, -3.6988],
        [-0.6969,  1.3062, -2.3532,  ..., -3.3773,  0.1456, -2.6532],
        ...,
        [ 1.0780, -0.1513, -1.7821,  ...,  0.0578, -1.1029, -1.6572],
        [ 0.2771,  0.7892, -2.2075,  ..., -0.0409, -0.1257,  0.0146],
        [-0.2808,  2.7156, -1.9834,  ..., -2.7026,  0.3808, -2.5764]],
       device='mps:0', grad_fn=<ViewBackward0>)


In [26]:
p = F.softmax(logits, dim=1)
print("p: ", p.shape, p)

p:  torch.Size([1024, 65]) tensor([[1.3630e-05, 5.2586e-05, 7.0520e-05,  ..., 1.5685e-05, 9.6518e-06,
         3.5213e-05],
        [7.1045e-05, 9.9767e-01, 1.0220e-05,  ..., 5.6453e-06, 1.6702e-06,
         8.2467e-07],
        [1.4207e-05, 1.0529e-04, 2.7112e-06,  ..., 9.7360e-07, 3.2988e-05,
         2.0086e-06],
        ...,
        [2.1209e-04, 6.2039e-05, 1.2145e-05,  ..., 7.6469e-05, 2.3954e-05,
         1.3761e-05],
        [4.0528e-05, 6.7635e-05, 3.3784e-06,  ..., 2.9488e-05, 2.7091e-05,
         3.1171e-05],
        [3.7916e-03, 7.5884e-02, 6.9091e-04,  ..., 3.3658e-04, 7.3478e-03,
         3.8182e-04]], device='mps:0', grad_fn=<SoftmaxBackward0>)


In [31]:
p[0]

tensor([1.3630e-05, 5.2586e-05, 7.0520e-05, 8.3644e-06, 9.6231e-06, 1.8873e-05,
        1.6621e-05, 1.9463e-05, 2.3391e-06, 1.1577e-05, 4.0147e-06, 5.8597e-06,
        2.1609e-06, 1.3190e-05, 3.1989e-05, 1.7757e-06, 1.0485e-06, 1.1934e-05,
        1.2338e-05, 1.2060e-06, 2.9203e-06, 2.6148e-06, 4.8507e-06, 2.0171e-06,
        5.4193e-05, 2.2877e-06, 2.8820e-05, 1.6604e-05, 2.6305e-05, 2.6218e-06,
        6.5262e-05, 2.2442e-05, 2.5957e-05, 5.4053e-05, 9.2774e-07, 1.4012e-06,
        5.0583e-06, 7.4192e-07, 1.0381e-05, 1.3249e-05, 2.5154e-06, 2.8848e-05,
        3.5969e-05, 6.1281e-05, 5.2438e-05, 6.0642e-05, 1.9346e-05, 4.7161e-05,
        1.0279e-06, 9.5057e-06, 6.2951e-05, 5.0819e-05, 9.9834e-01, 1.9206e-05,
        2.6520e-05, 5.9473e-06, 7.2602e-05, 8.1127e-05, 8.4929e-05, 2.8422e-05,
        1.8852e-04, 8.1447e-06, 1.5685e-05, 9.6518e-06, 3.5213e-05],
       device='mps:0', grad_fn=<SelectBackward0>)

In [24]:
p_max = torch.max(F.softmax(logits, dim=1), dim=1).values # ()
print("p_max: ", p_max.shape, p_max)

p_max:  torch.Size([1024]) tensor([0.9983, 0.9977, 0.9984,  ..., 0.9920, 0.9981, 0.1969], device='mps:0',
       grad_fn=<MaxBackward0>)


In [33]:
tau = torch.max(p,dim=1).indices
print("tau: ", tau.shape, tau)

tau:  torch.Size([1024]) tensor([52,  1, 58,  ..., 31, 47, 52], device='mps:0')
